In [ ]:
# =============================================================================
# SETUP
# =============================================================================
%load_ext autoreload
%autoreload 2
import os
import sys
from dotenv import load_dotenv

# Add project root to path
sys.path.append(os.path.abspath('..'))
load_dotenv()

print(f"✅ Setup Complete. Root: {os.path.abspath('..')}")

In [ ]:
# =============================================================================
# PASO 1: VALIDAR EL "MÚSCULO" (DynamoDB)
# =============================================================================
# Validamos neuro_agent/src/dynamo_client.py

try:
    from neuro_agent.src.adapters.dynamo_client import write_task_to_dynamo, fetch_context_from_dynamo
    
    USER_TEST_ID = "user_val_test"
    TASK_DESC = "Unit Test Task: Validate DynamoDB Client"
    
    print("\n--- 1. Testing Write ---")
    # write_task_to_dynamo is a @tool, so we invoke it or call it directly if possible.
    # It returns a string.
    # Since it is a @tool, it might need invocation via .run or as a callable if defined as such.
    # Let's check how @tool works. Usually it makes the function callable.
    try:
        if hasattr(write_task_to_dynamo, 'invoke'):
             # If it's a StructuredTool
             write_result = write_task_to_dynamo.invoke({'user_id': USER_TEST_ID, 'task_description': TASK_DESC})
        else:
             write_result = write_task_to_dynamo(USER_TEST_ID, TASK_DESC)
    except Exception as ie:
        # Fallback if arguments are positional but tool expects kwarg dictionary for invoke
         write_result = write_task_to_dynamo.invoke({'user_id': USER_TEST_ID, 'task_description': TASK_DESC})

    print(f"Write Result: {write_result}")
    
    print("\n--- 2. Testing Read ---")
    context = fetch_context_from_dynamo(USER_TEST_ID)
    print(f"Context Fetched: {context}")
    
    # Assert
    todos = context.get("todos", [])
    found = any(t.get('description') == TASK_DESC for t in todos)
    
    if found:
        print("\n✅ STEP 1 SUCCESS: Task written and retrieved from DynamoDB.")
    else:
        print("\n❌ STEP 1 FAILED: Task not found in fetched context.")
        
except ImportError as e:
    print(f"\n❌ IMPORT ERROR: {e}")
    print("Ensure neuro_agent/src/dynamo_client.py exists and is in the python path.")
except Exception as e:
    print(f"\n❌ EXECUTION ERROR: {e}")

In [ ]:
# =============================================================================
# PASO 2: VALIDAR EL "AGENTE IDIOTA" (Executor Handler)
# =============================================================================
import sys
import os

# Add neuro_agent to path to allow 'from src.dynamo_client import ...' inside lambda_executor.py
project_root = os.path.abspath('..')
neuro_agent_dir = os.path.join(project_root, 'neuro_agent')

if neuro_agent_dir not in sys.path:
    sys.path.append(neuro_agent_dir)
    print(f"Added {neuro_agent_dir} to sys.path")

try:
    # Import the handler. 'neuro_agent/deploy/lambda_executor.py' imports 'src.dynamo_client'.
    # 'src' is in 'neuro_agent'. So 'import src' works inside 'lambda_executor.py' because we added 'neuro_agent' to sys.path.
    # However, 'deploy' is also in 'neuro_agent'.
    # So 'from deploy.lambda_executor import lambda_handler' should work if we import relative to 'neuro_agent'.
    
    # Let's try importing as if we are in 'neuro_agent'.
    from deploy.lambda_executor import lambda_handler
    
    print("\n--- Testing Lambda Handler ---")
    USER_TEST_ID = "user_val_test"
    INSTRUCTION = "Execute unit test task: Buy milk via Lambda"
    
    mock_event = {
        "user_id": USER_TEST_ID,
        "explicit_instructions": INSTRUCTION
    }
    
    print(f"Invoking lambda_handler with event: {mock_event}")
    # Invoke synchronously
    response = lambda_handler(mock_event, None)
    print(f"Response: {response}")
    
    if response.get('statusCode') == 200:
        print("\n✅ STEP 2 SUCCESS: Executor Handler executed successfully and returned 200.")
    else:
        print(f"\n❌ STEP 2 FAILED: Response code {response.get('statusCode')}")

except ImportError as e:
    print(f"\n❌ IMPORT ERROR: {e}")
    print("Ensure 'neuro_agent' directory is correctly added to sys.path")
except Exception as e:
    print(f"\n❌ EXECUTION ERROR: {e}")